In [1]:
from catboost.datasets import msrank_10k

In [ ]:
train_df, test_df = msrank_10k()

X_train = train_df.drop([0, 1], axis=1).values
y_train = train_df[0].values

In [ ]:
y_train, y_train.reshape(-1,1)

In [ ]:
X_train, X_train[[0,1], :][:, [0,1]]

In [75]:
import numpy as np
import torch
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import StandardScaler

from typing import Tuple
import math

In [76]:
def compute_gain(y_value: float) -> float:
    return float(2 ** y_value - 1)

def dcg(ys_true: torch.Tensor, ys_pred: torch.Tensor,) -> float:
    _, argsort = torch.sort(ys_pred, descending=True, dim=0)
    ys_true_sorted = ys_true[argsort]
    ret = 0
    for idx, cur_y in enumerate(ys_true_sorted, 1):
        gain = compute_gain(cur_y)
        ret += gain / math.log2(idx + 1)
    return ret


def ndcg(ys_true: torch.Tensor, ys_pred: torch.Tensor) -> float:
    pred_dcg = dcg(ys_true, ys_pred)
    ideal_dcg = dcg(ys_true, ys_true)
    
    if ideal_dcg:
        return pred_dcg / ideal_dcg
    
    return 0

def compute_ideal_dcg(ys_true):
    return dcg(ys_true, ys_true)

In [77]:
def _compute_lambdas(y_true: torch.FloatTensor, y_pred: torch.FloatTensor) -> torch.FloatTensor:
    ideal_dcg = compute_ideal_dcg(y_true)
    N = 1 / ideal_dcg if ideal_dcg != 0 else 0
    
    # рассчитаем порядок документов согласно оценкам релевантности
    _, rank_order = torch.sort(y_true, descending=True, axis=0)
    rank_order += 1
    
    with torch.no_grad():
        # получаем все попарные разницы скоров в батче
        pos_pairs_score_diff = 1.0 + torch.exp(torch.Tensor(y_pred - y_pred.T))
        
        # поставим разметку для пар, 1 если первый документ релевантнее
        # -1 если второй документ релевантнее
        Sij = compute_labels_in_batch(y_true)
        # посчитаем изменение gain из-за перестановок
        gain_diff = compute_gain_diff(y_true)
        
        # посчитаем изменение знаменателей-дискаунтеров
        decay_diff = (1.0 / torch.log2(rank_order + 1.0)) - (1.0 / torch.log2(rank_order.T + 1.0))
        # посчитаем непосредственное изменение nDCG
        delta_ndcg = torch.abs(N * gain_diff * decay_diff)
        # посчитаем лямбды
        lambda_update =  (0.5 * (1 - Sij) - 1 / pos_pairs_score_diff) * delta_ndcg
        lambda_update = torch.sum(lambda_update, dim=1, keepdim=True)
        
        return lambda_update

    
def compute_labels_in_batch(y_true: torch.Tensor):
    
    # разница релевантностей каждого с каждым объектом
    rel_diff = y_true - y_true.T
    
    # 1 в этой матрице - объект более релевантен
    pos_pairs = (rel_diff > 0).type(torch.float32)
    
    # 1 тут - объект менее релевантен
    neg_pairs = (rel_diff < 0).type(torch.float32)
    Sij = pos_pairs - neg_pairs
    return Sij

def compute_gain_diff(y_true: torch.Tensor):
   return torch.pow(2.0, y_true) - torch.pow(2.0, y_true.T)

In [96]:
train_df, test_df = msrank_10k()

# train_df = train_df[train_df[1] == 1]
# test_df = test_df[test_df[1] == 13]

X_train = train_df.drop([0, 1], axis=1).values
y_train = train_df[0].values
query_ids_train = train_df[1].values.astype(int)

X_test = test_df.drop([0, 1], axis=1).values
y_test = test_df[0].values
query_ids_test = test_df[1].values.astype(int)

query_ids_test



array([  13,   13,   13, ..., 1303, 1318, 1318])

In [97]:
def _scale_features_in_query_groups(inp_feat_array: np.ndarray,
                                        inp_query_ids: np.ndarray) -> np.ndarray:

    for q in np.unique(inp_query_ids):
        idx = (inp_query_ids == q).nonzero()
        inp_feat_array[idx] = StandardScaler().fit_transform(inp_feat_array[idx])
    
    return inp_feat_array

In [98]:
X_train = torch.FloatTensor(_scale_features_in_query_groups(X_train, query_ids_train))
ys_train = torch.FloatTensor(y_train).reshape(-1,1)

X_test = torch.FloatTensor(_scale_features_in_query_groups(X_test, query_ids_test))
ys_test = torch.FloatTensor(y_test).reshape(-1,1)

X_train, ys_train

(tensor([[ 0.3161,  4.8171, -2.1759,  ..., -0.1118, -0.1959, -0.2662],
         [ 0.3161, -0.2350,  0.6171,  ..., -0.1118, -0.1959, -0.2662],
         [ 0.3161, -0.2350, -0.3139,  ..., -0.1118, -0.1959, -0.2662],
         ...,
         [ 0.4786, -0.4121,  1.3276,  ...,  0.0000, -0.1552, -0.3721],
         [ 0.4786, -0.4121, -0.0675,  ...,  0.0000, -0.1552, -0.3721],
         [ 0.4786,  1.7170, -0.0675,  ...,  0.0000, -0.1552, -0.3721]]),
 tensor([[2.],
         [2.],
         [0.],
         ...,
         [2.],
         [2.],
         [1.]]))

In [99]:
def gain(x):
     return 2 ** x - 1

def _ndcg_k(ys_true, ys_pred, ndcg_top_k) -> float:

        ys_pred_k = ys_pred[:ndcg_top_k]
        ys_true_k = ys_true[:ndcg_top_k]
        
        ys_pred_ind = torch.argsort(torch.Tensor(ys_pred_k), descending=True, dim=0)
        ys_true_cons_sorted = ys_true_k[ys_pred_ind]

        dcg_k = 0
        for i, v in enumerate(ys_true_cons_sorted):
            dcg_k += gain(v)/math.log2(i+2)


        ys_true_sorted, _ = torch.sort(ys_true_k, descending=True, dim=0)

        ideal_dcg_k = 0
        for i, v in enumerate(ys_true_sorted):
            ideal_dcg_k += gain(v)/math.log2(i+2)
        
        if ideal_dcg_k:
            return float(dcg_k / ideal_dcg_k)
        
        return 0.

def _calc_data_ndcg(queries_list: np.ndarray,
                        true_labels: torch.FloatTensor, preds: torch.FloatTensor) -> float:

    with torch.no_grad():
        ndcgs = []

        for q in np.unique(queries_list):
            idx = (queries_list == q).nonzero()
            batch_x = preds[idx]
            batch_y = true_labels[idx]
            
            ndcgs.append(_ndcg_k(batch_y, batch_x, ndcg_top_k=10))
        return np.mean(ndcgs)

In [107]:
from sklearn.linear_model import LinearRegression

In [103]:
subsamples = 0.3
colsample_bytree = 0.3

lr = 0.1
n_estimators = 100

trees = []
f_inds = []

def predict(data):
    res = torch.zeros_like(data[:,0].reshape(-1,1))
    for tree, f in zip(trees, f_inds):
        res -= lr * torch.FloatTensor(tree.predict(data[:,f])).reshape(-1,1)

    return res

In [108]:
def train_one_tree(cur_tree_idx: int,
                        train_preds: torch.FloatTensor
                        ) -> Tuple[DecisionTreeRegressor, np.ndarray]:
        
    sample_length = int(subsamples * X_train.size(0))
    colsample_bytree_length = int(colsample_bytree * X_train.size(1))

    query_lambdas = torch.zeros_like(ys_train)
    for q in np.unique(query_ids_train):
        idx = (q == query_ids_train).nonzero()[0]
        lambdas = _compute_lambdas(ys_train[idx], train_preds[idx])

        for i, l in enumerate(lambdas):
            query_lambdas[idx[i]] = l
    
    train_ind = torch.randperm(X_train.size(0))[:sample_length]
    feature_ind = torch.randperm(X_train.size(1))[:colsample_bytree_length]

    train = X_train[train_ind, :][:, feature_ind]
    tree = DecisionTreeRegressor(random_state=i, max_depth=5, min_samples_leaf=8)
    tree.fit(train, query_lambdas[train_ind])

    return tree, feature_ind

In [104]:
sample_length = int(subsamples * ys_train.size(0))
colsample_bytree_length = int(colsample_bytree * X_train.size(1))

train_preds = torch.zeros_like(ys_train)
for i in range(n_estimators):
    print(f'est. {i}')

    tree, f_ind = train_one_tree(i, train_preds)

    trees.append(tree)
    f_inds.append(f_ind)

    preds -= lr * torch.FloatTensor(tree.predict(X_train[:, feature_ind])).reshape(-1,1)
    
    ys_pred = predict(X_test)
    print(f'NDCG: {_calc_data_ndcg(query_ids_test, ys_test, ys_pred)}')
    


est. 0
NDCG: 0.5868012278594754
est. 1
NDCG: 0.6257973296398466
est. 2
NDCG: 0.6406590058044954
est. 3
NDCG: 0.6448476883498105
est. 4
NDCG: 0.6332731951366771
est. 5
NDCG: 0.6381299983371388
est. 6
NDCG: 0.6212337159297683
est. 7
NDCG: 0.624439345842058
est. 8
NDCG: 0.629209215329452
est. 9
NDCG: 0.6294910531829704
est. 10
NDCG: 0.6314435574141416
est. 11
NDCG: 0.6321063583547418
est. 12
NDCG: 0.6336471072652123
est. 13
NDCG: 0.6339108757674694
est. 14
NDCG: 0.6350872479379177
est. 15
NDCG: 0.6347528838298537
est. 16
NDCG: 0.6277794268998232
est. 17
NDCG: 0.6239526163447987
est. 18
NDCG: 0.6256190013479103
est. 19
NDCG: 0.6232688071375544
est. 20
NDCG: 0.6139760332351382
est. 21
NDCG: 0.6195109249515967
est. 22
NDCG: 0.6215319101783362
est. 23
NDCG: 0.6333920078521426
est. 24
NDCG: 0.6328004293821075
est. 25
NDCG: 0.6404187845235522
est. 26
NDCG: 0.6469623605636033
est. 27
NDCG: 0.6469752544706519
est. 28
NDCG: 0.6461223509501327
est. 29
NDCG: 0.6509703942997889
est. 30
NDCG: 0.648506

In [105]:
print(f'ndcg: {_calc_data_ndcg(query_ids_train, ys_train, preds)}')
preds[:10], ys_train[:10]

ndcg: 0.6715075127009688


(tensor([[ 0.0159],
         [ 0.9958],
         [-0.1382],
         [ 0.4877],
         [-0.0995],
         [-0.0645],
         [-0.1082],
         [ 0.2266],
         [ 0.2180],
         [ 0.3969]]),
 tensor([[2.],
         [2.],
         [0.],
         [2.],
         [1.],
         [1.],
         [1.],
         [2.],
         [1.],
         [0.]]))

In [106]:
res = torch.zeros_like(ys_test)
for tree, f in zip(trees, f_inds):
    res -= lr * torch.FloatTensor(tree.predict(X_test[:,f])).reshape(-1,1)

print(f'ndcg: {_calc_data_ndcg(query_ids_test, ys_test, res)}')
res[:10], ys_test[:10]

ndcg: 0.6361219127747145


(tensor([[ 0.0293],
         [-0.4662],
         [-0.2975],
         [ 0.2016],
         [-0.5488],
         [ 0.3072],
         [-0.2759],
         [-0.3473],
         [ 0.1107],
         [-0.0062]]),
 tensor([[2.],
         [1.],
         [3.],
         [1.],
         [0.],
         [0.],
         [1.],
         [0.],
         [0.],
         [2.]]))